# REDES NEURONALES PROFUNDAS
Utilice el conjunto de datos Fashion-MNIST para construir un clasificador convolucional de imágenes de productos. Para la construcción del modelo utilice los dos esquemas que se describen a continuación y compare los resultados:

1. Entrenar un Clasificador Convolucional de imágenes de productos basado en el Autocodificador convolucional de la práctica 1.
2. Clasificar las imágenes de productos utilizando un modelo de clasificación convolucional profundo.
3. Evalué el desempeño del clasificador en el conjunto de prueba y muestre la matriz de confusión.

Compruebe con ejemplos que el modelo es capaz de clasificar correctamente las imágenes de productos.

## OBJETIVO
Aplicar el proceso de aprendizaje a partir de datos para resolver problemas de clasificación utilizando redes neuronales convolucionales profundas sobre la herramienta Keras.

## DATOS
Incluidos en Keras.
También, existe otra fuente equivalente que se consigue en el siguiente URL https://www.kaggle.com/zalando-research/fashionmnist donde hay un resumen de estos datos en el archivo CVS y XLSX.

la clasificación para el aprendizaje supervisado es:

    Label 	Class
    0 	 	T-shirt/top
    1 	 	Trouser
    2 	 	Pullover
    3 	 	Dress
    4 	 	Coat
    5 	 	Sandal
    6 	 	Shirt
    7 	 	Sneaker
    8 	 	Bag
    9 	 	Ankle boot

**Importante: Lea los comentarios y apuntes del Notebook para tener claridad de los pasos.**
### Consideraciones
- Utilice sólo los conjuntos de datos indicado.
- El frameworks a utilizar es TensorFlow, Keras con Jupyter Notebbooks.

### Enlaces de interés
- documentación Keras, URL: https://keras.io/models/sequential/
- documentación TensorFlow, URL: https://www.tensorflow.org/versions
- Tutorial CNN basico, URL: https://www.kaggle.com/nhlr21/deep-keras-cnn-tutorial/notebook


## Importando Librerias

In [ ]:
# importando dependencias de trabajo
# importando librerias basicas
import os
import sys
import re
import gc

# importando modulos de analisis de datos, ML y graficas
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow import keras
from collections import Counter

# importando dependencias para tensorflow
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

# importando para sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import make_scorer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# importando para keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Conv2D
from keras.layers import MaxPooling2D

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Funciones Utiles

In [ ]:
# funcion que recibe una lista numpy y recupera la forma de cada elemento, devuelve una lista con formas
def get_shape(data):
    # respuesta de la funcion
    ans = list()

    for d in data:
        sp = d.shape
        ans.append(sp)
    return ans

In [ ]:
# funcion que transforma el entero de la clase a la palabra de la etiqueta, devuelve una lista de etiquetas
def class2label(data, labels):
    # respuesta de la funcion
    ans = list()

    for d in data:
        d = int(d)
        l = str(labels[d])
        ans.append(l)
    return ans

In [ ]:
# funcion que estandariza los datos en numpy de acuerdo a un valor min & max, devuelve un arreglo np flotante
def std_data(data, minv, maxv):
    rangev = maxv - minv
    ans = data.astype("float32")/float(rangev)
    # ans = pd.Series(ans)
    # respuesta de la funcion
    return ans

## Cargar y Preparar los Datos

Los pasos de esta seccion son:

1. Leer los datos desde MNIST.
2. Formatear los datos para que los acepte el DataFrame de Pandas.
2. Crear el DataFrame de Pandas con un esquema propio.
2. Formatear los datos MNIST para pobrar el DataFrame de pandas.
3. Revisar que todo este como deberia estar.

In [ ]:
# lista de nombres de las clasificaciones
label_names = [     # label number
    "T-shirt/top",      # 0
    "Trouser",          # 1
    "Pullover",         # 2
    "Dress",            # 3
    "Coat",             # 4
    "Sandal",           # 5
    "Shirt",            # 6
    "Sneaker",          # 7
    "Bag",              # 8
    "Ankle boot"        # 9
]

In [ ]:
# se carga el archivo de datos de trabajo por medio de Keras
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [ ]:
# nombres de columnas para el dataframe de pandas
col_names = [           # nombre de columna DF
    "img_data",         # datos de la imagen
    "img_shape",        # forma de la imagen
    "class",            # clase de la imagen, de 0 a 9
    "label",            # nombre de la clase
    "std_img_data",     # datos de la imagen estandarizados
    "cat_labels"        # etiquetas categoricas de la clase
]
# "ReshapeData", "Label", "Class", "DataSize", "ReshapeSize", "ResKeras", "ScoreKeras"]
# creando dataframe con columnas
fashion_df = pd.DataFrame(columns=col_names)

In [ ]:
# integrando datos de mnist
img_data = np.concatenate((x_train, x_test), axis=0)
class_data = np.concatenate((y_train, y_test), axis=0)
# recuperando forma de imagenes
img_shape = get_shape(img_data)
# recuperando etiquetas de las clases
labels = class2label(class_data, label_names)
# estandarizar los datos de la imagen
std_img_data = std_data(img_data, 0, 255)
# categorizando las clases a aprender
cat_labels = to_categorical(class_data, len(label_names))

In [ ]:
# cambio de formato para utilizar el dataframe
img_data = img_data.tolist()
std_img_data = std_img_data.tolist()
cat_labels = cat_labels.tolist()

In [ ]:
# definir arreglo basico de datos
data_lt = (
    img_data,       # datos de la imagen   0
    img_shape,      # forma de la imagen   1
    class_data,     # clase de la imagen   2
    labels,         # nombre de la clase   3
    std_img_data,   # datos de la imagen estandarizados 4
    cat_labels,     # etiquetas categoricas de la clase 5
)

In [ ]:
# poblamdo las columnas del dataframe
for col, data in zip(col_names, data_lt):
    fashion_df[col] = data

In [ ]:
fashion_df.info()

In [ ]:
# imprime el encabezado del dataframe
fashion_df.head()

In [ ]:
# libero memoria
gc.collect()

## Revisando los Datos
Es importante revisar los datos para saber que se esta trabajando con ellos. En esta sección se revisa la forma de los datos, se visualizan algunas imágenes y se revisa la distribución de las clases.

In [ ]:
# pruebo imagenes del dataset
# imagen de 5x5 subplots con tamaño de 10x10
sns.set_theme(style="white")
plt.figure(figsize=(10, 10))
# recorriendo las imagenes
for i in range(25):
    # creando subplots
    plt.subplot(5, 5, i+1)
    # xticks y yticks desactivados
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    # tomo la imagen de la columna de datos de la imagen
    # el mapa de colores es viridis de matplotlib
    # otras opciones son: "plasma", "viridis", "BuPu", "hsv", "Spectral"
    plt.imshow(fashion_df["img_data"][i],
               cmap=plt.cm.Spectral)
    # tomo el nombre de la clase de la columna de etiquetas
    plt.xlabel(fashion_df["label"][i])

## Preprocesar los Datos

Los pasos de esta seccion son:

1. Revisar que los datos esten bien.
2. Elegir la caracteristicas o propiedades de aprendizaje.
3. Elegir la variable objetivo del aprendizaje.
4. Dividir la conjunto de datos entre las poblaciones de entrenamiento y pruebas.
5. Formatear los datos de aprendizaje y objetivo acorde a la red neuronal.

In [ ]:
# cchequeo la distribucion de datos
plt.figure(figsize=(20, 8))
sns.set_theme(style="white")
sns.histplot(fashion_df[col_names[3]])
plt.show()

In [ ]:
# seleccionando caracteristicas de aprendizaje y variables objetivo
# recuperando la forma de las imagenes basado en el primer elemento de la lista

# recuperando los valores y ajustando el tensor para la CNN
A = fashion_df[col_names[4]]
# recuperando los valores de la cateogoria
b = fashion_df[col_names[5]].values

# fortateo de datos numpy
X = np.array([np.array(i, dtype="object") for i in A], dtype="object")
y = np.array([np.array(j, dtype="object") for j in b], dtype="object")

print(X.shape)
# forma basica general de las imagenes
imgsh = X[0].shape
# ajuste de forma para el modelo CNN
X = X.reshape(fashion_df.shape[0], imgsh[0], imgsh[1], 1)
print(X.shape)

In [ ]:
# semilla para el random
rseed = 42

# en tamanho de la muestra para pruebas esta entre 0.2 y 0.3
train_pop = 0.8
test_pop = 1.0 - train_pop

In [ ]:
# distribuir los datos entre entrenamiento vs. pruebas
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = test_pop, random_state = rseed)

In [ ]:
# formateo para keras y tensorflow
X_train = tf.convert_to_tensor(X_train, dtype="float64")
y_train = tf.convert_to_tensor(y_train, dtype="float64")
X_test = tf.convert_to_tensor(X_test, dtype="float64")
y_test = tf.convert_to_tensor(y_test, dtype="float64")
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

## Definir Modelo CNN

Los pasos de esta seccion son:

1. Definir las variables topologicas de la red neuronal.
2. Definir los parametros de optimizacion y aprendizaje del modelo.
3. Definir la topologia (capas) del modelo.
4. Definir las condiciones de entrenamiento para el modelo.

In [ ]:
# defino parametros necesarios para el modelo Autoencoder
# parametros para las capas
filters = 32    # 32, 64, 128
# TODO cambiar el numero de neuronas de la capa intermedia
# mid_neurons, v1 = 7*7, v2 = 28*28
mid_neurons = 7*7

# TODO cambiar la dimension del reshape convolucional
# mid_reshape, v1 = (7, 7, 1), v2 = (28, 28, 1)
mid_reshape = (7, 7, 1)
ksize = (3, 3)

# TODO cambiar el tamano del pool para reducir la imagen
# psize, v1 = (2, 2), v2 = (1, 1)
psize = (2, 2)

# numero de categorias en la salida
categories = len(label_names)
# numero de neuronas en la capa de salida
outn = categories

# funcion de activacion para las capas convolucionales
# otras opciones son: # "relu", "LeakyReLU"
act = "LeakyReLU"
# funcion de activacion para la capa de salida
# otras opciones son: # "sigmoid", "softmax"
out = "sigmoid"
# padding para las capas convolucionales
pad = "same"
# porcentaje de dropout para imagenes
ldrop = 0.2

# forma del kernel de entrada y de las capas intermedias
inshape = X[0].shape

# parametros de optimizacion del modelo
# funcion de perdida
# otras opciones son: "categorical_crossentropy"
l = "categorical_crossentropy"
# metodo de optimizacion
opti = "adam"
# metrica de evaluacion
met = ["accuracy"]

# parametros de la bitacora de entrenamiento del modelo, 1 es verbose
ver = 1
# numero de epocas
epo = 50
# tamano del lote de entrenamiento, se recomienda potencia de 2
# otras opciones son: 32, 64, 128, 256
bs = 128


In [ ]:
# definicion de las capas para el Autoencoder
layer_lt = (
    # capa de entrada
    Input(shape=inshape,
          name="LayIn"),

    # capa convolucional intermedia con regularizacion
    Conv2D(filters,
           ksize,
           activation=act,
           padding=pad,
           name="EnConv1"),
    MaxPooling2D(psize,
                 padding=pad,
                 name="EnPool1"),
    Dropout(ldrop,
            name="EnDrop1"),

    # capa convolucional intermedia con regularizacion
    Conv2D(int(filters/2),
           ksize,
           activation=act,
           padding=pad,
           name="EnConv2"),
    MaxPooling2D(psize,
                 padding=pad,
                 name="EnPool2"),
    Dropout(ldrop,
            name="EnDrop2"),

    # capa intermedia de 2D a 1D
    Flatten(name="LayFlat"),
    # capa intermedia densamente poblada con regularizacion
    Dense(mid_neurons,
          activation=act,
          name="DenseMid"),
    Dropout(ldrop,
            name="MidDrop"),
    
    # capa densa para clasificar los datos
    Dense(int(mid_neurons)//2,
          activation=act,
          name="ClsDense1"),
    Dropout(ldrop,
            name="ClsDrop1"),

    # capa densa para clasificar los datos
    Dense(int(mid_neurons)//4,
          activation=act,
          name="ClsDense2"),
    Dropout(ldrop,
            name="ClsDrop2"),

    # capa de salida
    Dense(outn,
          activation=out,
          name="LayOut")
)

In [ ]:
# definiendo el modelo CNN en Keras
cnn_model = Sequential(layer_lt)
cnn_model.model_name="DCNN_Classifier"

In [ ]:
# compilando las condiciones de optimizacion y ajuste del Modelo CNN
cnn_model.compile(loss=l,
                  optimizer=opti,
                  metrics=met)

In [ ]:
# resumen de la topologia del modelo
cnn_model.summary()

In [ ]:
# imprimo imagen de la topologia del modelo
wdir = os.getcwd()
folder_models = "Models"
model_fn = "discws_cnn_classifier.png"
model_fp = os.path.join(folder_models, model_fn)
print("La imagen del modelo esta en:", model_fp)
keras.utils.plot_model(cnn_model,
                       model_fp,
                       show_shapes=True)

In [ ]:
# condiciones de parada temprana
cnn_earlystop_acc = EarlyStopping(monitor="val_accuracy",
                                  min_delta=0.001,
                                  patience=5,
                                  verbose=ver,
                                  mode="max",
                                  restore_best_weights=True)

## Entrenar Modelo

Los pasos de esta seccion son:

1. Entrenar el modelo con el conjunto de entrenamineto.

In [ ]:
# ajustando el modelo MLP Keras
cnn_log = cnn_model.fit(
    x=X_train,    # np.array(X_trainB),
    y=y_train,    # to_categorical(np.array(y_trainB), categories),
    batch_size=bs,
    epochs=epo,
    verbose=ver,
    callbacks=[cnn_earlystop_acc],
    workers=8,
    shuffle=False,
    use_multiprocessing=True,
    validation_data=(X_test, y_test),
)

## Probar Modelo

Los pasos de esta seccion son:

1. Probar el modelo con el conjunto de pruebas.
2. Evaluar globalmente los resultados.
3. Guardar el modelo entrenado.

In [ ]:
cnn_eval = cnn_model.evaluate(x=X_test,
                              y=y_test)

In [ ]:
# resultados generales
print("Perdida promedio: ", cnn_eval[0])
print("Precision promedio: ", cnn_eval[1])

In [ ]:
# pruebas sobre el modelo
cnn_predictions = cnn_model.predict(X_test,
                                    verbose=ver)
print(cnn_predictions.shape)

In [ ]:
# guardar el modelo entrenado
wdir = os.getcwd()
folder_models = "Models"
model_fn = "discws_cnn_classifier.h5"
model_fp = os.path.join(folder_models, model_fn)
print("El modelo entrenado esta en:", model_fp)
cnn_model.save(model_fp)
# tf.keras.models.save_model(cnn_autoencoder, model_fpn)

## Mostrar Resultados

Los pasos de esta sección son:

1. Mostrar las curvas de aprendizaje.
2. Mostrar la Matriz de confusión del Clasificador.
3. Mostrar la clasificación del modelo.
4. Mostrar la abstracción del Clasificador.

In [ ]:
# ajuste de las predicciones para ver el reporte de matrix de confusion
cnn_count = np.array(cnn_predictions).argmax(axis=1)
y_count = np.array(y_test).argmax(axis=1)
print(cnn_count.shape)
print(y_count.shape)

In [ ]:
# preparando informe de la matriz de confusion
cnn_counter = Counter(cnn_count)
cnn_matrix = confusion_matrix(y_count, cnn_count)
cnn_report = classification_report(y_count, cnn_count)

In [ ]:
# Informe por consola de las pruebas para el clasificador
print("----- Reporte de Pruebas para el clasificador CNN -----")
print("--- Conteo ---\n", cnn_counter)
print("--- Matriz de Confusion ---\n", cnn_matrix)
print("--- Reporte de Pruebas: ---")
print(cnn_report)
print("--- Puntaje General---\n")
print(" - Perdida: ", cnn_eval[0])
print(" - Precision: ", cnn_eval[1])

In [ ]:
# despliegue grafico de la matriz de confusion
# dataframe para graficar
confusion_df = pd.DataFrame(data=cnn_matrix,
                            columns=label_names,
                            index=label_names)

# creando el reporte para graficar
cnn_report = classification_report(y_count,
                                   cnn_count,
                                   output_dict=True)
# dataframe del repodrte
report_df = pd.DataFrame(data=cnn_report)

# dataframe para desplegar el puntaje de las clases
cols = [
    "0",
    "1",
    "2",
    "3",
    "4",
    "5",
    "6",
    "7",
    "8",
    "9"
]
idx = [
    "precision",
    "recall",
    "f1-score"
]
class_report = pd.DataFrame(report_df[cols],
                            index=idx)
class_report = pd.DataFrame(class_report.values,
                            columns=label_names,
                            index=idx)
class_report = class_report.T

# dataframe para desplegar el reporte global
cols = [
    "accuracy",
    "macro avg",
    "weighted avg"
]
macro_report = pd.DataFrame(report_df[cols],
                            index=idx)
macro_report = macro_report.T

In [ ]:
report_df.head()

In [ ]:
# mapa de calor de la matriz de confusion por conteo
plt.figure(figsize=(20, 20))
sns.set_theme(style="white", font_scale=1.6)
sns.heatmap(confusion_df, annot=True, fmt='.1f', cmap="BuPu")
plt.show()

In [ ]:
# presicion, recall y f1-score de las clases en el modelo
plt.figure(figsize=(10, 10))
sns.set_theme(style="white", font_scale=1.6)
sns.heatmap(class_report, annot=True, fmt='.2f', cmap="Blues")
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
sns.set_theme(style="white", font_scale=1.6)
sns.heatmap(macro_report, annot=True, fmt='.2f', cmap="Greens")
plt.show()

In [ ]:

# reporte del aprendizaje
# base de la figura
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# datos de la figura en de perdida y precision
ax1.plot(cnn_log.history["loss"], 'red', label="Train Loss")
ax1.plot(cnn_log.history["val_loss"], 'darkorange', label="Test Loss")
ax2.plot(cnn_log.history["accuracy"], 'red', label="Train Accuracy")
ax2.plot(cnn_log.history["val_accuracy"], 'royalblue', label="Test Accuracy")

# leyenda de la grafica
fig.suptitle("LEARNING BEHAVIOR")
ax1.grid(True)
ax2.grid(True)
ax1.set_title("Loss")
ax2.set_title("Accuracy")
ax1.set(xlabel="Epoch [cycle]", ylabel="loss [%]")
ax2.set(xlabel="Epoch [cycle]", ylabel="Acc [%]")
fig.legend()
fig.show()

In [ ]:
# prueba funcional del autoencoder
max_img = 10
random_test_img = np.random.randint(len(X_test),
                                    size=max_img)

In [ ]:
classprob_df = pd.DataFrame(columns=random_test_img,
                            index=label_names)

for img in random_test_img:
    temp_cls = np.array(cnn_predictions[img])
    classprob_df[img] = temp_cls

In [ ]:
middle_layer = "LayFlat"
cnn_abstraction = Model(inputs=cnn_model.input,
                        outputs=cnn_model.get_layer(middle_layer).output)

In [ ]:
# desplegando pruebas
sns.set_theme(style="white")
plt.figure(figsize=(20, 10))
og_shape = fashion_df[col_names[1]][0]
og_label = fashion_df[col_names[3]][0]

for i, img in enumerate(random_test_img):
    # imagen original
    ax = plt.subplot(3, max_img, i + 1)
    temp_X = np.array(X_test[img])
    # imprimir el nombre de la imagen
    plt.title(img)
    # ajustar ty tamaño y color al original
    plt.imshow(temp_X.reshape(og_shape),
               cmap=plt.cm.Spectral)
    # imprimir el nombre de la clase
    idx = list(y_test[img]).index(max(y_test[img]))
    # print(idx, label_names[idx])
    # print(label_names)
    plt.xlabel(label_names[idx])
    plt.xticks([])
    plt.yticks([])
    
    # imagen abstracta de la capa intermedia del autoencoder
    ax = plt.subplot(3, max_img, max_img + i + 1)
    # recuperar la imagen abstracta
    temp_abstract = cnn_abstraction(temp_X)
    # formato de la imagen abstracta
    temp_abstract = np.array(temp_abstract)
    # ajusta tamaño y color
    px = temp_abstract.shape[0]*temp_abstract.shape[1]
    px = int(np.sqrt(px))
    plt.imshow(temp_abstract.reshape(px, px),
               cmap=plt.cm.viridis)
    # imprimir nombre de la clase
    plt.xlabel(label_names[idx])


    # clasificacion aproximada
    temp_cls = np.array(cnn_predictions[img])
    temp_cls = temp_cls.argmax(axis=0)
    temp_cls = int(temp_cls)
    temp_cls = label_names[temp_cls]
# plt.show()

plt.figure(figsize=(30,30))
sns.set_theme(style="white", font_scale=1.8)
sns.heatmap(classprob_df, annot=True, fmt='.3f', cmap="BuPu")
plt.tick_params(axis='both',
                which='major',
                labelbottom = False,
                bottom=False,
                top=False,
                labeltop=True)
plt.show()